### **Librerías**

In [15]:
# pendientes
# si en vez de enviar a whatsapp por persona se les envia a whatsapp grupal por seller?

In [16]:
import os
import math
import time
import smtplib

import numpy as np
import pandas as pd

from email.mime.application import MIMEApplication
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

from google.cloud import bigquery

### **BigQuery**

In [17]:
# Proyecto y cliente
project_id='bi-fcom-drmb-local-pe-sbx'
client = bigquery.Client(project=project_id)

# Configuración job
job_config = bigquery.QueryJobConfig()
job_config.query_parameters = (bigquery.ScalarQueryParameter("limit", "INT64", 1000),)

query = """
    SELECT * FROM `bi-fcom-drmb-local-pe-sbx.Dragonite_SX_KPIs.reporte_backlog`
"""

# Query execution
query_job = client.query(query, job_config=job_config)
query_job.result()
data = query_job.to_dataframe()

c:\Users\gfloress\AppData\Local\Programs\Python\Python39\lib\site-packages\google\auth\_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


### **Variables**

In [18]:
# Correo

correo_usuario = 'sleona@falabella.com'
contrasena = 'Ingreso2024$$'
asuntos_base = [
    'F.COM | 🚨 HOY ESTÁS EN RUTA 🚨', 
    'F.COM | 🚨 HOY ESTÁS EN RUTA 🚨',
    'F.COM | 🚨 GESTIONA TUS ÓRDENES PENDIENTES 🚨',
    'F.COM | 🚨 GESTIONA TUS ÓRDENES PENDIENTES 🚨',
    'F.COM | 🚨 ENTREGA TUS PEDIDOS HOY 🚨',
    'F.COM | 🚨 ENTREGA TUS PEDIDOS HOY 🚨'
]

# Columnas

columnas = ['sellerName', 'trackingCode','deliveryOrderNumber', 'sku', 'description', 'promisedBySeller', 'Estatusenvio']

# Variables iniciales


### **Funciones**

In [19]:
"""
def send_email(correo_seller, html_message, seller_email, seller_name, seller_id, seller_phone, seller_link, sender_email, sender_password):
    # Set up the SMTP server
    server = smtplib.SMTP('smtp.office365.com', 587)
    server.starttls()

    # Log in to the SMTP server
    server.login(correo_usuario, contrasena)

    # Set up the message
    msg = MIMEMultipart()
    msg['From'] = 'fcomlogisticasx@falabella.com'
    msg['To'] = correo_seller
    msg['Subject'] = asunto
    msg['CC'] = 'soporteperufcom@falabella.com'

    # Attach the HTML message
    msg.attach(MIMEText(html_message, 'html'))

    # Send the email
    server.sendmail(sender_email, seller_email, msg.as_string())

    # Close the SMTP connection
    server.quit()
"""

"\ndef send_email(correo_seller, html_message, seller_email, seller_name, seller_id, seller_phone, seller_link, sender_email, sender_password):\n    # Set up the SMTP server\n    server = smtplib.SMTP('smtp.office365.com', 587)\n    server.starttls()\n\n    # Log in to the SMTP server\n    server.login(correo_usuario, contrasena)\n\n    # Set up the message\n    msg = MIMEMultipart()\n    msg['From'] = 'fcomlogisticasx@falabella.com'\n    msg['To'] = correo_seller\n    msg['Subject'] = asunto\n    msg['CC'] = 'soporteperufcom@falabella.com'\n\n    # Attach the HTML message\n    msg.attach(MIMEText(html_message, 'html'))\n\n    # Send the email\n    server.sendmail(sender_email, seller_email, msg.as_string())\n\n    # Close the SMTP connection\n    server.quit()\n"

### **Mensajes**

### **Data**

In [20]:
# Leer el registro desde un archivo Excel
data = data.query("sellerEmail.notnull()").query("sellerEmail.str.contains('@')")

### **Condicionales**

In [7]:
# Correos

## 1. AAA que no son Dropoff
data_1 = (
            data
            .query("segmento == 'AAA' and "
                   "accion not in ['No accionar', 'Cancelar', 'Cambio de estado', 'Comunicar envío día siguiente'] and "
                   "carrierCode != 'dropoff'")
        )

## 2. Other Sellers (Hoy estas en Ruta, Retraso & Pending)
data_2 = (
            data
            .query("(segmento != 'AAA') and "
                   "(Speech == 'Speech_2') and "
                   "(accion not in ['No accionar', 'Cancelar', 'Cambio de estado', 'Comunicar envío día siguiente']) and "
                   "(carrierCode != 'dropoff')")
        )

## 3. Other Sellers (Hoy estas en Ruta y Retraso)
data_3 = (
            data
            .query("(segmento != 'AAA') and "
                   "(Speech == 'Speech_3') and "
                   "(accion not in ['No accionar', 'Cancelar', 'Cambio de estado', 'Comunicar envío día siguiente']) and "
                   "(carrierCode != 'dropoff')")
        )

## 4. Other Sellers (Only Pending)
data_4 = (
            data
            .query("(segmento != 'AAA') and "
                   "(Speech == 'Speech_4') and "
                   "(accion not in ['No accionar', 'Cancelar', 'Cambio de estado', 'Comunicar envío día siguiente'])")
        )

## 5. Other Sellers (Drop Off)
data_6 = (
            data
            .query("(segmento != 'AAA') and "
                   "(carrierCode == 'dropoff') and "
                   "(Estatusenvio in ['A tiempo', '1 dia de retraso', '2 dias de retraso', '3 dias de retraso']) and "
                   "(accion not in ['No accionar', 'Cancelar', 'Cambio de estado', 'Comunicar envío día siguiente'])")
        )

## 6. AAA (Drop Off)
data_7 = (
            data
            .query("(segmento == 'AAA') and "
                   "(carrierCode == 'dropoff') and "
                   "(lastStatusGSC != 'pending') and "
                   "(Estatusenvio in ['A tiempo', '1 dia de retraso', '2 dias de retraso', '3 dias de retraso']) and "
                   "(accion not in ['No accionar', 'Cancelar', 'Cambio de estado', 'Comunicar envío día siguiente'])")
        )

# WhatsApp

## 1. AAA (con retraso)
data_5 = (
            data
            .query("(segmento == 'AAA') and "
                   "(Estatusenvio in ['1 dia de retraso', '2 dias de retraso', '3 dias de retraso']) and "
                   "(accion not in ['No accionar', 'Cancelar', 'Cambio de estado', 'Comunicar envío día siguiente']) and "
                   "(carrierCode != 'dropoff')")
        )

### **Envío de correos**

#### **1. AAA**

In [8]:
# Establecer la conexión con el servidor SMTP de Outlook
server = smtplib.SMTP('smtp.office365.com', 587)
server.starttls()

# Iniciar sesión en el servidor SMTP
server.login(correo_usuario, contrasena)

# Iterar sobre las tiendas únicas en el DataFrame
for seller in data_1['sellerId'].unique():
    
    # Filtrar dataframe por tienda
    df_seller = data_1[data_1['sellerId'] == seller]

    # Obtener datos por tienda
    correo_seller = df_seller['sellerEmail'].iloc[0]
    phone_asignado = df_seller['loyaltyPhone'].iloc[0]
    link_phone = df_seller['EnlaceAAA'].iloc[0]
    sellerName = df_seller['sellerName'].iloc[0]

    # Seleccionar columnas específicas para mostrar en el cuerpo del correo
    df_seller =  (
                    df_seller[columnas]
                    .drop_duplicates()
                    .sort_values(by='Estatusenvio', ascending=False)
                    .rename(columns={
                              'sellerName': 'Nombre del seller',
                              'deliveryOrderNumber': 'Número de orden',
                              'trackingCode': 'Rastreo',
                              'description': 'Descripción',
                              'promisedBySeller': 'Fecha de promesa',
                              'Estatusenvio': 'Estado del envío'
                    })
                  )
    
    # Configurar el mensaje
    msg = MIMEMultipart()
    msg['From'] = 'fcomlogisticasx@falabella.com'
    msg['To'] = correo_seller
    msg['Subject'] = asuntos_base[0].format(seller)
    msg['CC'] = 'soporteperufcom@falabella.com'

    # Crear el cuerpo del correo en formato HTML
    mensaje_html = f"""
      <html>
        <head>
          <style>
            table {{
              border-collapse: collapse;
              width: 100%;
            }}
            th, td {{
              border: 1px solid black;
              padding: 8px;
              text-align: left;
            }}
            th {{
              background-color: orange;
              color: white;
              font-size: 16px;
              font-weight: bold;
              text-align: center;
            }}
          </style>
        </head>
        <body>
          <p><h2><strong>Hola {sellerName}, con ID {seller}!</strong> Estas son las órdenes que recolectaremos hoy: </h2></p>
          
          <br>
          {df_seller.to_html(index=False)}
          <br>
          <p>Podrás revisar tus órdenes en falabella seller center desde <a href= "https://sellercenter.falabella.com/order">aquí</a>.</p>
          <p>Para gestionar tus pedidos a tiempo es importante:</p>
              <ol>
                  <li>Imprimir y pegar la <em>etiqueta</em> en el exterior de tu empaque. Se visualiza al lado izquierdo del número de orden como <em>Label</em>.</li>
                  <li>Hacer clic en <em>Listo para despachar</em> - como máximo un día antes de la fecha límite (antes de las 10:00PM).</li>
              </ol>
              
          <p>🚚 Revisa nuestro tutorial de despacho: <a href="https://youtu.be/lpk1_y9uYDg">Tutorial de despacho</a>.</p>
              
          <p>Siguiendo estos pasos estaremos programando la recolección de tus órdenes en la fecha límite de despacho. Recuerda que el tiempo de espera del operador logístico es de 10 minutos. </p>

          <p>🌟 Si necesitas soporte personalizado a través de WhatsApp, contáctanos en el {phone_asignado} o <a href="{link_phone}">ingresando al siguiente link</a>.</p>
          
          <p>¡Gracias!</p> 
        </body>
      </html>
    """

    # Adjuntar el cuerpo del correo en formato HTML
    msg.attach(MIMEText(mensaje_html, 'html'))

    # Enviar el correo electrónico
    server.sendmail(correo_usuario, correo_seller, msg.as_string())

# Cerrar la conexión con el servidor SMTP
server.quit()

(221, b'2.0.0 Service closing transmission channel')

#### **2. Others sellers (mañana estas en ruta y retraso) & pending**

In [9]:
# Convertir todas las columnas a cadenas para evitar el error de 'float' object has no attribute 'encode'
data_2= data_2.map(lambda x: str(x))

# Establecer la conexión con el servidor SMTP de Outlook
server = smtplib.SMTP('smtp.office365.com', 587)
server.starttls()

# Iniciar sesión en el servidor SMTP
server.login(correo_usuario, contrasena)

# Iterar sobre las tiendas únicas en el DataFrame
for seller in data_2['sellerId'].unique():
    
    # Filtrar el DataFrame para la tienda actual
    df_seller = data_2[data_2['sellerId'] == seller]

    # Obtener datos por tienda
    correo_seller = df_seller['sellerEmail'].iloc[0]
    phone_asignado = df_seller['loyaltyPhone'].iloc[0]
    link_phone = df_seller['EnlaceAAA'].iloc[0]
    sellerName = df_seller['sellerName'].iloc[0]

    # Seleccionar columnas específicas para mostrar en el cuerpo del correo
    df_seller =  (
                    df_seller[columnas]
                    .drop_duplicates()
                    .sort_values(by='Estatusenvio', ascending=False)
                    .rename(columns={
                              'sellerName': 'Nombre del seller',
                              'deliveryOrderNumber': 'Número de orden',
                              'trackingCode': 'Rastreo',
                              'description': 'Descripción',
                              'promisedBySeller': 'Fecha de promesa',
                              'Estatusenvio': 'Estado del envío'
                    })
                  )

    # Configurar el mensaje
    msg = MIMEMultipart()
    msg['From'] = 'fcomlogisticasx@falabella.com'
    msg['To'] = correo_seller
    msg['Subject'] = asuntos_base[1].format(seller)
    msg['CC'] = 'soporteperufcom@falabella.com'

    # Crear el cuerpo del correo en formato HTML
    mensaje_html = f"""
      <html>
        <head>
          <style>
            table {{
              border-collapse: collapse;
              width: 100%;
            }}
            th, td {{
              border: 1px solid black;
              padding: 8px;
              text-align: left;
            }}
            th {{
              background-color: orange;
              color: white;
              font-size: 16px;
              font-weight: bold;
              text-align: center;
            }}
          </style>
        </head>
        <body>
          <p><h2><strong>Hola {sellerName}, con ID {seller}!</strong> Estas son las órdenes que recolectaremos hoy  </h2></p></p>
          
          <br>
          {df_seller.to_html(index=False)}
          <br>
          <p>*Tener en cuenta que si tienes ordenes con días de retraso ya están impactando el cumplimiento de tus indicadores. Recuerda que tu nivel de cumplimiento se relaciona directamente con la cantidad cobrada por cofinanciamiento logístico. Si necesitas más detalles sobre estos cobros, haz clic aqui :<a href="https://www.falabellasellers.com/comisiones-y-costos">https://www.falabellasellers.com/comisiones-y-costos</a></li></p>
          <p>*Las ordenes con 3 días de retraso de no ser entregadas serán canceladas el día de hoy.</p>
          <br>

          <p><h2>🚨 🚨 Por otro lado, detectamos que tienes <strong>órdenes pendientes</strong> por colocar "listo para despachar"</h2></p>
          <p>Recuerda que debes cambiar el estado de estas órdenes a Listo para despachar un día previo de la fecha límite de despacho, como máximo hasta las 10 pm, para que podamos incluirlas en la ruta de recolección</p>
          
          <p>Queremos reforzar contigo el proceso para gestionarlas:</p>
          <ol>
              <li>Podrás revisar tus órdenes en falabella seller center desde <a href= "https://sellercenter.falabella.com/order">aquí</a>.</li>
              <li>Imprimir y pegar la <em>etiqueta</em> en el exterior de tu empaque. Se visualiza al lado izquierdo del número de orden como <em>Label</em>.</li>
              <li>Hacer clic en <em>Listo para despachar</em> - como máximo un día antes de la fecha límite (antes de las 10:00PM).</li>
          </ol>
          <p>🚚 Revisa nuestro tutorial de despacho: <a href="https://youtu.be/lpk1_y9uYDg">Tutorial de despacho</a>.</p>
          
          <p>¡No pierdas ventas! Recuerda que si la orden se cancela no habrá lugar a reclamo.</p>

          <p>Siguiendo estos pasos estaremos programando la recolección de estos pedidos para el día siguiente. Recuerda que, al llegar a tu almacén, el tiempo de espera del operador logístico es de 10 minutos.</p>
          
          <p><strong>¡Importante!</strong> </p>
              <li>Las órdenes con retraso impactarán en tu indicador de envíos a tiempo afectando tu puntaje operacional. Considera que el cofinanciamiento logístico varía de acuerdo a tu desempeño. Para más información sobre haz clic <a href= "https://www.falabellasellers.com/comisiones-y-costos">aquí</a>.</li>
              <li>Las ordenes con 3 días de retraso de no ser entregadas el día de hoy,serán canceladas.</li>
              <li>Este correo es un refuerzo. Siempre debes entregar los pedidos gestionados a tiempo en su respectiva fecha límite de despacho.</li>
          
          <p>¡Contamos con tu apoyo para mejorar la experiencia de nuestros clientes!</p>
              
        </body>
      </html>
    """

    # Adjuntar el cuerpo del correo en formato HTML
    msg.attach(MIMEText(mensaje_html, 'html'))

    # Enviar el correo electrónico
    server.sendmail(correo_usuario, msg['To'], msg.as_string())

# Cerrar la conexión con el servidor SMTP
server.quit()

(221, b'2.0.0 Service closing transmission channel')

#### **3. Others sellers (mañana estas en ruta y retraso)**

In [10]:
# Convertir todas las columnas a cadenas para evitar el error de 'float' object has no attribute 'encode'
data_4 = data_4.map(lambda x: str(x))

# Configurar la conexión SMTP fuera del bucle
server = smtplib.SMTP('smtp.office365.com', 587)
server.starttls()
server.login(correo_usuario, contrasena)

# Función para enviar correos electrónicos
def enviar_correo(correo_destinatario, mensaje):
    try:
        # Enviar el correo electrónico
        server.sendmail(correo_usuario, correo_destinatario, mensaje.as_string())
        print("Correo electrónico enviado exitosamente.")
    except Exception as e:
        print(f"Error al enviar el correo electrónico: {e}")

# Iterar sobre las tiendas únicas en el DataFrame
for seller in data_4['sellerId'].unique():
    
    # Filtrar el DataFrame para la tienda actual
    df_seller = data_4[data_4['sellerId'] == seller]

    # Obtener datos por tienda
    correo_seller = df_seller['sellerEmail'].iloc[0]
    phone_asignado = df_seller['loyaltyPhone'].iloc[0]
    link_phone = df_seller['EnlaceAAA'].iloc[0]
    sellerName = df_seller['sellerName'].iloc[0]

    # Seleccionar columnas específicas para mostrar en el cuerpo del correo
    df_seller = (
                  df_seller[columnas]
                  .drop_duplicates()
                  .rename(columns={
                      'sellerName': 'Nombre del seller',
                      'deliveryOrderNumber': 'Número de orden',
                      'trackingCode': 'Rastreo',
                      'description': 'Descripción',
                      'promisedBySeller': 'Fecha de promesa',
                      'Estatusenvio': 'Estado del envío'
                  })
                )

    # Configurar el mensaje
    msg = MIMEMultipart()
    msg['From'] = 'fcomlogisticasx@falabella.com'
    msg['To'] = correo_seller
    msg['Subject'] = asuntos_base[2].format(seller)
    msg['CC'] = 'soporteperufcom@falabella.com'

    # Crear el cuerpo del correo en formato HTML
    mensaje_html = f"""
      <html>
        <head>
          <style>
            table {{
              border-collapse: collapse;
              width: 100%;
            }}
            th, td {{
              border: 1px solid black;
              padding: 8px;
              text-align: left;
            }}
            th {{
              background-color: orange;
              color: white;
              font-size: 16px;
              font-weight: bold;
              text-align: center;
            }}
          </style>
        </head>
        <body>
          <p><h2><strong>Hola {sellerName}, con ID {seller}! Te recordamos que algunos pedidos aún no están marcados como "Listo para despachar"!</strong></h2></p>
          <p>Por favor, confirma antes de las 10:00 pm de HOY para evitar problemas para el cliente y mantener tu rendimiento.</p>
          <p>Recuerda que debes realizar este cambio para que la(s) órden(es) sean (programadas) para su recolección en la fecha límite de despacho.</p>
          <br>
          {df_seller.to_html(index=False)}
          <br>
          <p><strong>Podrás revisar tus órdenes en falabella seller center desde <a href= "https://sellercenter.falabella.com/order">aquí</a>.</strong></p>
          <p>Este correo es un refuerzo. Siempre debes entregar los pedidos gestionados a tiempo en su respectiva fecha límite de despacho.</p>
          <p>¡Contamos con tu apoyo para mejorar la experiencia de nuestros clientes!</p>
        </body>
      </html>
    """

    # Adjuntar el cuerpo del correo en formato HTML
    msg.attach(MIMEText(mensaje_html, 'html'))

    # Enviar el correo electrónico llamando a la función definida anteriormente
    enviar_correo(correo_seller, msg)

# Cerrar la conexión SMTP después de enviar todos los correos electrónicos
server.quit()

Correo electrónico enviado exitosamente.
Correo electrónico enviado exitosamente.
Correo electrónico enviado exitosamente.
Correo electrónico enviado exitosamente.
Correo electrónico enviado exitosamente.
Correo electrónico enviado exitosamente.
Correo electrónico enviado exitosamente.
Correo electrónico enviado exitosamente.
Correo electrónico enviado exitosamente.
Correo electrónico enviado exitosamente.
Correo electrónico enviado exitosamente.
Correo electrónico enviado exitosamente.
Correo electrónico enviado exitosamente.
Correo electrónico enviado exitosamente.
Correo electrónico enviado exitosamente.
Correo electrónico enviado exitosamente.
Correo electrónico enviado exitosamente.
Correo electrónico enviado exitosamente.
Correo electrónico enviado exitosamente.
Correo electrónico enviado exitosamente.
Correo electrónico enviado exitosamente.
Correo electrónico enviado exitosamente.
Correo electrónico enviado exitosamente.
Correo electrónico enviado exitosamente.
Correo electróni

(221, b'2.0.0 Service closing transmission channel')

#### **4. Only pendings**

In [11]:
# Convertir todas las columnas a cadenas para evitar el error de 'float' object has no attribute 'encode'
data_4= data_4.map(lambda x: str(x))

# Establecer la conexión con el servidor SMTP de Outlook
server = smtplib.SMTP('smtp.office365.com', 587)
server.starttls()

# Iniciar sesión en el servidor SMTP
server.login(correo_usuario, contrasena)

# Iterar sobre las tiendas únicas en el DataFrame
for seller in data_4['sellerId'].unique():
    
    # Filtrar el DataFrame para la tienda actual
    df_seller = data_4[data_4['sellerId'] == seller]

    # Obtener datos por tienda
    correo_seller = df_seller['sellerEmail'].iloc[0]
    phone_asignado = df_seller['loyaltyPhone'].iloc[0]
    link_phone = df_seller['EnlaceAAA'].iloc[0]
    sellerName = df_seller['sellerName'].iloc[0]

    # Seleccionar columnas específicas para mostrar en el cuerpo del correo
    df_seller = (
                  df_seller[columnas]
                  .drop_duplicates()
                  .sort_values(by='Estatusenvio', ascending=False)
                  .rename(columns={
                      'sellerName': 'Nombre del seller',
                      'deliveryOrderNumber': 'Número de orden',
                      'trackingCode': 'Rastreo',
                      'description': 'Descripción',
                      'promisedBySeller': 'Fecha de promesa',
                      'Estatusenvio': 'Estado del envío'
                  })
                )

    # Configurar el mensaje
    msg = MIMEMultipart()
    msg['From'] = 'fcomlogisticasx@falabella.com'
    msg['To'] = correo_seller
    msg['Subject'] = asuntos_base[3].format(seller)
    msg['CC'] = 'soporteperufcom@falabella.com'

    # Crear el cuerpo del correo en formato HTML
    mensaje_html = f"""
      <html>
        <head>
          <style>
            table {{
              border-collapse: collapse;
              width: 100%;
            }}
            th, td {{
              border: 1px solid black;
              padding: 8px;
              text-align: left;
            }}
            th {{
              background-color: orange;
              color: white;
              font-size: 16px;
              font-weight: bold;
              text-align: center;
            }}
          </style>
        </head>
        <body>
          <p><h2><strong>Hola {sellerName}, con ID {seller}! NO has colocado el “listo para despachar” </strong> en las siguientes órdenes:</h2></p></p>
          
          <br>
          {df_seller.to_html(index=False)}
          <br>

          <p>Queremos reforzar contigo el proceso para gestionarlas:</p>
          <ol>
              <li>Podrás revisar tus órdenes en falabella seller center desde <a href= "https://sellercenter.falabella.com/order">aquí</a>.</li>
              <li>Imprimir y pegar la <em>etiqueta</em> en el exterior de tu empaque. Se visualiza al lado izquierdo del número de orden como <em>Label</em>.</li>
              <li>Hacer clic en <em>Listo para despachar</em> - como máximo un día antes de la fecha límite (antes de las 10:00PM).</li>
          </ol>
          
          <p>🚚 Revisa nuestro tutorial de despacho: <a href="https://youtu.be/lpk1_y9uYDg">Tutorial de despacho</a>.</p>
          
          
          <p><strong>¡Importante!</strong> </p>
          <p>Es importante entregar todos los pedidos en su respectiva fecha límite de despacho, evita el retraso en tus entregas.</p>
              <li>Las órdenes con retraso impactarán en tu indicador de envíos a tiempo afectando tu puntaje operacional. Considera que el cofinanciamiento logístico varía de acuerdo a tu desempeño. Para más información sobre haz clic <a href= "https://www.falabellasellers.com/comisiones-y-costos">aquí</a>.</li>
              <li>Las ordenes con 3 días de retraso de no ser entregadas el día de hoy,serán canceladas.</li>
              
          <p>Este correo es un refuerzo. Siempre debes entregar los pedidos gestionados a tiempo en su respectiva fecha límite de despacho.</p>
          
          <p>¡Contamos con tu apoyo para mejorar la experiencia de nuestros clientes!</p>
              
        </body>
      </html>
    """

    # Adjuntar el cuerpo del correo en formato HTML
    msg.attach(MIMEText(mensaje_html, 'html'))

    # Enviar el correo electrónico
    server.sendmail(correo_usuario, msg['To'], msg.as_string())

# Cerrar la conexión con el servidor SMTP
server.quit()

(221, b'2.0.0 Service closing transmission channel')

#### **5. Dropoff a Others**

In [12]:
# Convertir todas las columnas a cadenas para evitar el error de 'float' object has no attribute 'encode'
data_6= data_6.map(lambda x: str(x))

# Establecer la conexión con el servidor SMTP de Outlook
server = smtplib.SMTP('smtp.office365.com', 587)
server.starttls()

# Iniciar sesión en el servidor SMTP
server.login(correo_usuario, contrasena)

# Iterar sobre las tiendas únicas en el DataFrame
for seller in data_6['sellerId'].unique():
    
    # Filtrar el DataFrame para la tienda actual
    df_seller = data_6[data_6['sellerId'] == seller]

    # Obtener datos por tienda
    correo_seller = df_seller['sellerEmail'].iloc[0]
    phone_asignado = df_seller['loyaltyPhone'].iloc[0]
    link_phone = df_seller['EnlaceAAA'].iloc[0]
    sellerName = df_seller['sellerName'].iloc[0]
    
    punto_dropoff = df_seller['punto_dropoff'].iloc[0]
    direccion =df_seller['direccion_dropoff'].iloc[0]
    tienda = df_seller['nombre_dropoff'].iloc[0]
    horario = df_seller['horario_dropoff'].iloc[0]
    frecuencia = df_seller['frecuencia_dropoff'].iloc[0] 

    # Seleccionar columnas específicas para mostrar en el cuerpo del correo
    df_seller = (
                  df_seller[columnas]
                  .drop_duplicates()
                  .sort_values(by='Estatusenvio', ascending=False)
                  .rename(columns={
                      'sellerName': 'Nombre del seller',
                      'deliveryOrderNumber': 'Número de orden',
                      'trackingCode': 'Rastreo',
                      'description': 'Descripción',
                      'promisedBySeller': 'Fecha de promesa',
                      'Estatusenvio': 'Estado del envío'
                  })
                )

    # Configurar el mensaje
    msg = MIMEMultipart()
    msg['From'] = 'fcomlogisticasx@falabella.com'
    msg['To'] = correo_seller
    msg['Subject'] = asuntos_base[4].format(seller)
    msg['CC'] = 'soporteperufcom@falabella.com'

    # Crear el cuerpo del correo en formato HTML
    mensaje_html = f"""
      <html>
        <head>
          <style>
            table {{
              border-collapse: collapse;
              width: 100%;
            }}
            th, td {{
              border: 1px solid black;
              padding: 8px;
              text-align: left;
            }}
            th {{
              background-color: orange;
              color: white;
              font-size: 16px;
              font-weight: bold;
              text-align: center;
            }}
          </style>
        </head>
        <body>
          <p><h2><strong>Hola {sellerName}, con ID {seller}! Estas son las órdenes que debes entregar hoy en tu punto de acopio asignado:</strong></h2></p></p>
          
          <br>
          {df_seller.to_html(index=False)}
          <br>

          <strong><p>Datos sobre tu punto de acopio:</strong></p>
          <ol>
              <li>Nombre del punto: {punto_dropoff}</li>
              <li>Tienda: {tienda}</li>
              <li>Dirección: {direccion}</li>
              <li>Horario de entrega: {horario}</li>
              <li>Frecuencia: {frecuencia}</li>
          </ol>
          
          <p>🚚 Revisa nuestro tutorial de despacho: <a href="https://youtu.be/lpk1_y9uYDg">Tutorial de despacho</a>.</p>
          
          
          <p><strong>¡Importante!</strong> </p>
              <li>Recuerda que el punto de acopio debe firmar el cargo de entrega que le proporciones.</li>
              <li>Las ordenes con 3 días de retraso de no ser entregadas el día de hoy,serán canceladas.</li>
              <li>Este correo es un refuerzo. Siempre debes entregar los pedidos gestionados a tiempo en su respectiva fecha límite de despacho.</li>
              
          
          <p>¡Contamos con tu apoyo para mejorar la experiencia de nuestros clientes!</p>
              
        </body>
      </html>
    """

    # Adjuntar el cuerpo del correo en formato HTML
    msg.attach(MIMEText(mensaje_html, 'html'))

    # Enviar el correo electrónico
    server.sendmail(correo_usuario, msg['To'], msg.as_string())

# Cerrar la conexión con el servidor SMTP
server.quit()

(221, b'2.0.0 Service closing transmission channel')

#### **6. AAA Dropoff**

In [13]:
# Convertir todas las columnas a cadenas para evitar el error de 'float' object has no attribute 'encode'
data_7= data_7.map(lambda x: str(x))

# Establecer la conexión con el servidor SMTP de Outlook
server = smtplib.SMTP('smtp.office365.com', 587)
server.starttls()

# Iniciar sesión en el servidor SMTP
server.login(correo_usuario, contrasena)

# Iterar sobre las tiendas únicas en el DataFrame
for seller in data_7['sellerId'].unique():
    
    # Filtrar el DataFrame para la tienda actual
    df_seller = data_7[data_7['sellerId'] == seller]

    # Obtener datos por tienda
    correo_seller = df_seller['sellerEmail'].iloc[0]
    phone_asignado = df_seller['loyaltyPhone'].iloc[0]
    link_phone = df_seller['EnlaceAAA'].iloc[0]
    sellerName = df_seller['sellerName'].iloc[0]
    
    punto_dropoff = df_seller['punto_dropoff'].iloc[0]
    direccion =df_seller['direccion_dropoff'].iloc[0]
    tienda = df_seller['nombre_dropoff'].iloc[0]
    horario = df_seller['horario_dropoff'].iloc[0]
    frecuencia = df_seller['frecuencia_dropoff'].iloc[0] 

    # Seleccionar columnas específicas para mostrar en el cuerpo del correo
    df_seller = (
                  df_seller[columnas]
                  .drop_duplicates()
                  .sort_values(by='Estatusenvio', ascending=False)
                  .rename(columns={
                      'sellerName': 'Nombre del seller',
                      'deliveryOrderNumber': 'Número de orden',
                      'trackingCode': 'Rastreo',
                      'description': 'Descripción',
                      'promisedBySeller': 'Fecha de promesa',
                      'Estatusenvio': 'Estado del envío'
                  })
                )

    # Configurar el mensaje
    msg = MIMEMultipart()
    msg['From'] = 'fcomlogisticasx@falabella.com'
    msg['To'] = correo_seller
    msg['Subject'] = asuntos_base[5].format(seller)
    msg['CC'] = 'soporteperufcom@falabella.com'

    # Crear el cuerpo del correo en formato HTML
    mensaje_html = f"""
      <html>
        <head>
          <style>
            table {{
              border-collapse: collapse;
              width: 100%;
            }}
            th, td {{
              border: 1px solid black;
              padding: 8px;
              text-align: left;
            }}
            th {{
              background-color: orange;
              color: white;
              font-size: 16px;
              font-weight: bold;
              text-align: center;
            }}
          </style>
        </head>
        <body>
          <p><h2><strong>Hola {sellerName}, con ID {seller}! Estas son las órdenes que debes entregar hoy en tu punto de acopio asignado:</strong></h2></p></p>
          
          <br>
          {df_seller.to_html(index=False)}
          <br>

          <strong><p>Datos sobre tu punto de acopio:</strong></p>
          <ol>
              <li>Nombre del punto: {punto_dropoff}</li>
              <li>Tienda: {tienda}</li>
              <li>-	Dirección: {direccion}</li>
              <li>Horario de entrega: {horario}</li>
              <li>Frecuencia: {frecuencia}</li>
          </ol>
          
          <p>🚚 Revisa nuestro tutorial de despacho: <a href="https://youtu.be/lpk1_y9uYDg">Tutorial de despacho</a>.</p>
          
          
          <p><strong>¡Importante!</strong> </p>
              <li>Recuerda que el punto de acopio debe firmar el cargo de entrega que le proporciones.</li>
              <li>Las ordenes con 3 días de retraso de no ser entregadas el día de hoy,serán canceladas.</li>
              <li>Este correo es un refuerzo. Siempre debes entregar los pedidos gestionados a tiempo en su respectiva fecha límite de despacho.</li>
              
          <p><p>🌟 Si necesitas soporte personalizado a través de WhatsApp, contáctanos en el {phone_asignado} o <a href="{link_phone}">ingresando al siguiente link</a>.</p>
          <p>¡Contamos con tu apoyo para mejorar la experiencia de nuestros clientes!</p>
              
        </body>
      </html>
    """

    # Adjuntar el cuerpo del correo en formato HTML
    msg.attach(MIMEText(mensaje_html, 'html'))

    # Enviar el correo electrónico
    server.sendmail(correo_usuario, msg['To'], msg.as_string())

# Cerrar la conexión con el servidor SMTP
server.quit()

(221, b'2.0.0 Service closing transmission channel')

### **Envío de Whatsapp**

#### **1. AAA**

In [14]:
import pyautogui
import threading
import pywhatkit

In [196]:
def send_whatsapp_message(seller_data):
    mensaje_whatsapp = """
        *Hola {sellerName}, Hoy estás en la ruta de recolección*

        Detectamos que tienes órdenes poseen retraso en la entrega, es importante que las entregues hoy al operador logístico.

        Para mayor detalle te hemos enviado un mail al correo *{correo}* con asunto "F.COM | 🚨 HOY ESTAS EN RUTA🚨"

        🚚 Aquí podrás revisar el tutorial de gestión de órdenes: https://youtu.be/lpk1_y9uYDg

        Queremos reforzar contigo el proceso para gestionarlas:

        📦 Para que podamos recolectar tu orden deberás:

        1-	Podrás revisar tus órdenes en falabella seller center desde aquí: https://sellercenter.falabella.com/order
        2-	Recuerda hacer clic en “Listo para despachar” como máximo un día antes de la fecha límite de despacho (antes de las 10p.m.)

        🌟Si necesitas soporte personalizado a través de WhatsApp, contáctanos en el *{Phone}* o haz clic aquí {link_phone}

        ¡Gracias!
    """
    whatsapp_message = mensaje_whatsapp.format(
        sellerName=seller_data['sellerName'],
        correo=seller_data['sellerEmail'],
        Phone=seller_data['sellerPhone'],
        link_phone=seller_data['loyaltyPhone'],
    )
    
    pywhatkit.sendwhatmsg_instantly(seller_data['sellerPhone'], whatsapp_message)

In [197]:
lista = data_5.groupby(['sellerId'],dropna=False).agg(orders=('deliveryOrderNumber','count')).reset_index().sort_values(['orders'], ascending=False)
sellers_lista = lista[lista['orders']>=5]['sellerId'].to_list()
data_5_final = data_5[data_5['sellerId'].isin(sellers_lista)]

In [198]:
data_5_final['sellerId'].drop_duplicates().count()

29

In [199]:
# Batch processing
threads = []
for seller_id, seller_data in data_5_final.groupby('sellerId'):
    seller_data = seller_data.iloc[0]  # Assuming each seller has the same data
    thread = threading.Thread(target=send_whatsapp_message, args=(seller_data,))
    threads.append(thread)
    thread.start()
    time.sleep(12)  # Add delay between starting threads

# Wait for all threads to finish
for thread in threads:
    thread.join()